In [1]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.model.supervised import Supervised
from nlptools.utils import Config

cfg = Config("../config/cornell.yml")
model = Supervised.build(cfg)

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


data/cornell/model.pt {'skill_name': 'cornell', 'bert_model_name': None, 'encoder_hidden_layers': 3, 'encoder_attention_heads': 8, 'encoder_intermediate_size': 2048, 'encoder_hidden_size': 512, 'dropout': 0.1, 'decoder_hidden_layers': 3, 'decoder_attention_heads': 8, 'decoder_hidden_size': 2048, 'beam_size': 1, 'vocab_size': 30522, 'pad_id': 103, 'bos_id': 101, 'eos_id': 102, 'unk_id': 100}
hiii 2048


2019-05-15 21:14:25,359 | INFO: Optimizer: sgd with learning_rate: 0.0001


In [2]:
model.skill.model.config

{'decoder_hidden_layers': 3,
 'decoder_attention_heads': 8,
 'decoder_hidden_size': 2048,
 'bert_model_name': None,
 'vocab_size': 30522,
 'encoder_hidden_layers': 3,
 'encoder_attention_heads': 8,
 'max_position_embeddings': 512,
 'encoder_intermediate_size': 2048,
 'encoder_hidden_size': 512}

In [1]:
dialog = iter(model.generator).next()
dialog.to("cuda:0")
response = dialog["response_cornell"].data
utterance = dialog["utterance"].data
utterance_mask = dialog["utterance_mask"].data
sentiment = dialog["sentiment"].data
encoder_out = model.skill.model.dialog_embedding(utterance, utterance_mask, sentiment)
prev_output = response.data[:, :-1]
target_output = response.data[:, 1:]
print(utterance.shape)
print(utterance_mask.shape)
print(encoder_out.shape)
print(prev_output.shape)

NameError: name 'model' is not defined

In [4]:
for name, parameter in model.skill.model.named_parameters():
    print(name, parameter.requires_grad)

encoder.encoder.embeddings.word_embeddings.weight True
encoder.encoder.embeddings.position_embeddings.weight True
encoder.encoder.embeddings.token_type_embeddings.weight True
encoder.encoder.embeddings.LayerNorm.weight True
encoder.encoder.embeddings.LayerNorm.bias True
encoder.encoder.encoder.layer.0.attention.self.query.weight True
encoder.encoder.encoder.layer.0.attention.self.query.bias True
encoder.encoder.encoder.layer.0.attention.self.key.weight True
encoder.encoder.encoder.layer.0.attention.self.key.bias True
encoder.encoder.encoder.layer.0.attention.self.value.weight True
encoder.encoder.encoder.layer.0.attention.self.value.bias True
encoder.encoder.encoder.layer.0.attention.output.dense.weight True
encoder.encoder.encoder.layer.0.attention.output.dense.bias True
encoder.encoder.encoder.layer.0.attention.output.LayerNorm.weight True
encoder.encoder.encoder.layer.0.attention.output.LayerNorm.bias True
encoder.encoder.encoder.layer.0.intermediate.dense.weight True
encoder.encode

In [7]:
for parameter in model.skill.model.decoder.fc3.parameters():
    print(parameter.requires_grad)

False


In [4]:
target_output = target_output.unsqueeze(-1).contiguous().view(-1)
target_output.shape

torch.Size([1596])

In [7]:
import nlptools.zoo.encoders.transformer
import importlib
importlib.reload(nlptools.zoo.encoders.transformer)
from nlptools.zoo.encoders.transformer import TransformerDecoder
embedding = model.skill.model.encoder.embedding
decoder_hidden_layers = 1
decoder_attention_heads = 3
decoder_hidden_size = 3072
dropout = 0
decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout)
decoder = decoder.to("cuda:0")
output = decoder(prev_output, encoder_out, utterance_mask)
output.shape

torch.Size([84, 19, 30522])

In [8]:
decoder

TransformerDecoder(
  (word_embedding): Embedding(30522, 768, padding_idx=0)
  (position_embedding): Embedding(512, 768)
  (layer_norm): BertLayerNorm()
  (layers): ModuleList(
    (0): TransformerDecoderLayer(
      (self_attn): MultiheadAttention(
        (linear_layers): ModuleList(
          (0): Linear(in_features=768, out_features=768, bias=True)
          (1): Linear(in_features=768, out_features=768, bias=True)
          (2): Linear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0)
        (output_linear): Linear(in_features=768, out_features=768, bias=True)
      )
      (encoder_attn): MultiheadAttention(
        (linear_layers): ModuleList(
          (0): Linear(in_features=768, out_features=768, bias=True)
          (1): Linear(in_features=768, out_features=768, bias=True)
          (2): Linear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0)
        (output_linear): Linear(in_features=768, out_fea